# OTP Verification System

In [2]:
import random
import smtplib
import re
from tkinter import Tk, Label, Entry, Button, messagebox
from time import time

class OTPVerificationSystem:
    def __init__(self):
        self.sender_email = 'sender@gmail.com'
        self.attempts = 0
        self.max_attempts = 3
        self.otp_expiration_time = 600  # 10 minutes
        self.otp = None
        self.otp_expiration = None

    def generate_otp(self):
        return ''.join([str(random.randint(0, 9)) for _ in range(6)])

    def send_otp_email(self, receiver_email, otp):
        subject = "OTP Verification"
        body = f"Your OTP is: {otp}"

        try:
            server = smtplib.SMTP('smtp.gmail.com', 587)
            server.starttls()
            server.login(self.sender_email, 'ikba luvi dowz viki')

            message = f"From: {self.sender_email}\nTo: {receiver_email}\nSubject: {subject}\n\n{body}"
            server.sendmail(self.sender_email, receiver_email, message)
            messagebox.showinfo("OTP Sent", "OTP sent successfully to your email.")
            server.quit()
        except Exception as e:
            messagebox.showerror("Error", f"Error: {e}")

    def validate_email(self, email):
        pattern = re.compile(r"^[a-zA-Z0-9+_.-]+@[a-zA-Z0-9.-]+$")
        return re.match(pattern, email)

    def verify_otp(self, entered_otp):
        if self.otp_expiration is None or time() > self.otp_expiration:
            return "OTP expired"  
        elif self.otp != entered_otp:
            self.attempts += 1
            if self.attempts >= self.max_attempts:
                return "Max attempts reached"
            return "Incorrect OTP"
        else:
            return "Success"

    def resend_otp(self):
        email = email_entry.get()

        if not self.validate_email(email):
            messagebox.showerror("Error", "Invalid email format. Please enter a valid email address.")
            return

        self.otp = self.generate_otp()
        self.otp_expiration = time() + self.otp_expiration_time  
        self.attempts = 0

        self.send_otp_email(email, self.otp)

    def verify(self):
        entered_otp = otp_entry.get()

        if entered_otp == "":
            messagebox.showerror("Error", "Please enter the OTP.")
            return

        verification_result = self.verify_otp(entered_otp)
        if verification_result == "Success":
            if self.attempts >= self.max_attempts:
                messagebox.showerror("Error", "Max attempts reached. Access denied.")
            else:
                messagebox.showinfo("Success", "OTP verification successful. Access granted!")
        elif verification_result == "OTP expired":
            messagebox.showerror("Error", "OTP expired. Access denied.")
        elif verification_result == "Max attempts reached":
            messagebox.showerror("Error", "Max attempts reached. Access denied.")
        else:
            messagebox.showerror("Error", "Incorrect OTP. Access denied.")

if __name__ == "__main__":
    otp_system = OTPVerificationSystem()

    root = Tk()
    root.title("OTP Verification System")

    Label(root, text="Enter your email:").grid(row=0, column=0, padx=10, pady=5)
    email_entry = Entry(root)
    email_entry.grid(row=0, column=1, padx=10, pady=5)

    Label(root, text="Enter OTP:").grid(row=1, column=0, padx=10, pady=5)
    otp_entry = Entry(root)
    otp_entry.grid(row=1, column=1, padx=10, pady=5)

    Button(root, text="Verify OTP", command=otp_system.verify).grid(row=2, column=0, columnspan=2, pady=10)
    Button(root, text="Send OTP", command=otp_system.resend_otp).grid(row=3, column=0, columnspan=2, pady=5)

    root.mainloop()